In [1]:
import os
import yaml
import sys
from pathlib import Path
import csv
import json 
PROJECT_ROOT = Path.cwd().parent  # points to nl-pe/
SRC_PATH = PROJECT_ROOT / "src"
sys.path.insert(0, str(SRC_PATH))  # make nl_pe importable
os.chdir(PROJECT_ROOT)  

from dotenv import load_dotenv
from nl_pe.llm import LLM_CLASSES
from nl_pe.llm.prompter import Prompter

In [2]:
CONFIG_PATH = "configs/llm/config.yaml"
with open(CONFIG_PATH, "r") as f:
    config = yaml.safe_load(f)


load_dotenv()
prompter = Prompter(config)

In [3]:
# Prompt with the template

template_path = 'hotel_gen.jinja2'
response = prompter.prompt_from_temp(template_path)

print("Full response:", response)

2025-09-25 15:01:25,595 - OpenAILLM - DEBUG - calling llm api
2025-09-25 15:01:34,631 - OpenAILLM - DEBUG - Successfully parsed LLM response as JSON object
Full response: {'message': '{\n  "doc_1": "Nestled amidst the breathtaking landscapes of the Dolomites in Italy, the Alpen Glow Mountain Lodge offers an exquisite escape for adventure seekers and tranquility lovers alike. This luxury lodge provides guests with lavish suites featuring panoramic mountain views and private balconies. The award-winning on-site restaurant serves gourmet Alpine cuisine with locally sourced ingredients. After a thrilling day on the slopes, unwind in the spa’s thermal baths or cozy up by the grand stone fireplace in the rustic-chic lounge. A blend of traditional charm and modern elegance, Alpen Glow promises an unforgettable experience, showcasing the serene beauty of the Italian Alps.",\n  "doc_2": "Located in the heart of vibrant Hanoi, Vietnam, the Ha Noi Budget Boutique Hotel is perfect for travelers se

In [ ]:
import json
import csv
from pathlib import Path

# Assuming `response` is the JSON response you received
if 'JSON_dict' in response:
    json_data = response['JSON_dict']

    # Extract docs from doc_1, doc_2, ... format
    doc_list = []
    i = 1
    while True:
        doc_key = f'doc_{i}'
        if doc_key in json_data:
            doc_list.append(json_data[doc_key])
            i += 1
        else:
            break

    print(f"Successfully parsed JSON with {len(doc_list)} documents in doc_1-doc_{i-1} format")

    # Save to CSV
    output_dir = Path('data/gen_docs')
    output_dir.mkdir(parents=True, exist_ok=True)

    csv_path = output_dir / 'hotels_1.csv'
    with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['d_id', 'd_text'])
        for idx, doc_text in enumerate(doc_list, 1):
            writer.writerow([idx, doc_text])

    print(f"CSV file created successfully: {csv_path}")
else:
    print("No JSON_dict found in response")


Successfully parsed JSON with 2 documents in doc_1-doc_100 format
CSV file created successfully: data\gen_docs\hotels_1.csv
Total documents saved: 2
